# 13-02 认证与授权

JWT、OAuth2、RBAC 权限控制。

## 1. JWT 认证完整实现

In [ ]:
import jwt from 'jsonwebtoken';
import bcrypt from 'bcrypt';

const JWT_SECRET = process.env.JWT_SECRET;
const REFRESH_SECRET = process.env.REFRESH_SECRET;

class AuthService {
  // 登录
  async login(email, password) {
    const user = await this.findUser(email);
    if (!user) throw new Error('用户不存在');
    
    const valid = await bcrypt.compare(password, user.passwordHash);
    if (!valid) throw new Error('密码错误');
    
    return this.generateTokens(user);
  }
  
  // 生成 Token 对
  generateTokens(user) {
    const accessToken = jwt.sign(
      { userId: user.id, role: user.role },
      JWT_SECRET,
      { expiresIn: '15m' }  // 15分钟过期
    );
    
    const refreshToken = jwt.sign(
      { userId: user.id, type: 'refresh' },
      REFRESH_SECRET,
      { expiresIn: '7d' }  // 7天过期
    );
    
    return { accessToken, refreshToken };
  }
  
  // 验证 Access Token
  verifyAccessToken(token) {
    try {
      return jwt.verify(token, JWT_SECRET);
    } catch {
      return null;
    }
  }
  
  // 刷新 Token
  async refresh(refreshToken) {
    try {
      const payload = jwt.verify(refreshToken, REFRESH_SECRET);
      const user = await this.findUserById(payload.userId);
      return this.generateTokens(user);
    } catch {
      throw new Error('Invalid refresh token');
    }
  }
}

## 2. RBAC 权限控制

In [ ]:
// 基于角色的访问控制
const ROLES = {
  ADMIN: ['read', 'write', 'delete', 'manage_users'],
  USER: ['read', 'write'],
  GUEST: ['read']
};

function checkPermission(userRole, requiredPermission) {
  const permissions = ROLES[userRole] || [];
  return permissions.includes(requiredPermission);
}

// Express 中间件
function requirePermission(permission) {
  return (req, res, next) => {
    if (!checkPermission(req.user.role, permission)) {
      return res.status(403).json({ error: 'Forbidden' });
    }
    next();
  };
}

// 使用
app.delete('/users/:id',
  authenticate,
  requirePermission('delete'),
  deleteUserHandler
);

## 3. API 安全最佳实践

In [ ]:
// 安全中间件示例
import helmet from 'helmet';
import rateLimit from 'express-rate-limit';
import cors from 'cors';

// 安全响应头
app.use(helmet());

// CORS 配置
app.use(cors({
  origin: process.env.ALLOWED_ORIGINS?.split(',') || 'http://localhost:3000',
  credentials: true
}));

// 速率限制
const limiter = rateLimit({
  windowMs: 15 * 60 * 1000,  // 15分钟
  max: 100,  // 限制100次请求
  message: 'Too many requests'
});
app.use('/api/', limiter);

// 认证路由更严格
const authLimiter = rateLimit({
  windowMs: 15 * 60 * 1000,
  max: 5,  // 登录尝试限制
  skipSuccessfulRequests: true
});
app.use('/api/auth/login', authLimiter);

## 练习

1. 实现完整的用户注册/登录/登出系统
2. 添加密码强度检查和重置功能
3. 实现 API Key 认证机制